**Imports**

In [1]:
# Essentials
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import os
import googleapiclient.discovery
# Import functions for data preprocessing & data preparation
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string
from string import punctuation
import nltk
import re

In [2]:
API_KEY="yourapikey"

In [3]:
API_SERVICE_NAME = "youtube"
API_VERSION = "v3"

In [4]:
VIDEO_ID="LAOoF2gyQaA"

In [5]:
youtube = googleapiclient.discovery.build(
    API_SERVICE_NAME, API_VERSION, developerKey = API_KEY)

request = youtube.commentThreads().list(
    part="snippet,replies",
    videoId=VIDEO_ID,
    maxResults=100,
)
response = request.execute()

In [6]:
response['items'][0]['snippet']['topLevelComment']['snippet']['textOriginal']

'The Roast of Alec Baldwin premieres September 15 on Comedy Central. See who will be roasting Alec here: \nhttps://www.youtube.com/watch?v=ePp-8vOmmdk'

In [7]:
df=pd.json_normalize(response['items'])

In [8]:
df.head()

,kind,etag,id,snippet.channelId,snippet.videoId,snippet.topLevelComment.kind,snippet.topLevelComment.etag,snippet.topLevelComment.id,snippet.topLevelComment.snippet.channelId,snippet.topLevelComment.snippet.videoId,...,snippet.topLevelComment.snippet.authorChannelId.value,snippet.topLevelComment.snippet.canRate,snippet.topLevelComment.snippet.viewerRating,snippet.topLevelComment.snippet.likeCount,snippet.topLevelComment.snippet.publishedAt,snippet.topLevelComment.snippet.updatedAt,snippet.canReply,snippet.totalReplyCount,snippet.isPublic,replies.comments
0,youtube#commentThread,3mBUGIip4cJzBs1R2FJcoAoD_-A,UgyiVIVM7XBnRFfhvr54AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,youtube#comment,vslFAPPvV2pErpmlic8EO_MInuQ,UgyiVIVM7XBnRFfhvr54AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,...,UCUsN5ZwHx2kILm84-jPDeXw,True,none,10393,2019-09-03T18:43:15Z,2019-09-03T18:43:15Z,True,92,True,"[{'kind': 'youtube#comment', 'etag': '4J3UO6Ud..."
1,youtube#commentThread,UDsqMzCGORU_qhUh-xVRrI773TU,UgzOzDX1jT_2jCzSkVp4AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,youtube#comment,ItUdfEZO9OQVD37DJBPSoh1WOQ4,UgzOzDX1jT_2jCzSkVp4AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,...,UC61bOqWqm7n713umLk8_6fQ,True,none,0,2023-11-26T14:27:20Z,2023-11-26T14:27:53Z,True,0,True,NaN
2,youtube#commentThread,KA3N0MGLE3SD711iH19SPWqQmI4,Ugxkt5ag1bZQT4H1Lkd4AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,youtube#comment,dMcc5qXlNuRfWO3kvqhMGZoaJHU,Ugxkt5ag1bZQT4H1Lkd4AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,...,UC9fJlw2kDrpO0K-tnTr96YQ,True,none,0,2023-11-24T02:04:02Z,2023-11-24T02:04:02Z,True,0,True,NaN
3,youtube#commentThread,hWbSgNujOAquGc2wt685Xv7Ueho,UgxS9faNwb01M7cLvGl4AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,youtube#comment,ZMBw3ZeRC9uAbEGOqBleF1UlVcs,UgxS9faNwb01M7cLvGl4AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,...,UCStbrsZziGZXtvyxvQzMjfQ,True,none,0,2023-11-22T08:43:29Z,2023-11-22T08:43:29Z,True,0,True,NaN
4,youtube#commentThread,ilfrCMU3RUUDcY9nlvJ63Y-JqmY,UgwWJsvEnp0HNoqezCV4AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,youtube#comment,FuEi4FmXmvory_qNhHIfelcUU7E,UgwWJsvEnp0HNoqezCV4AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,...,UCkBAUYfxGnockoTDMC3l46g,True,none,0,2023-11-20T10:15:29Z,2023-11-20T10:15:29Z,True,0,True,NaN


In [9]:
df.columns=df.columns.str.removeprefix('snippet.topLevelComment.').str.removeprefix('snippet.').str.removesuffix('.value')
df.columns

Index(['kind', 'etag', 'id', 'channelId', 'videoId', 'kind', 'etag', 'id',
       'channelId', 'videoId', 'textDisplay', 'textOriginal',
       'authorDisplayName', 'authorProfileImageUrl', 'authorChannelUrl',
       'authorChannelId', 'canRate', 'viewerRating', 'likeCount',
       'publishedAt', 'updatedAt', 'canReply', 'totalReplyCount', 'isPublic',
       'replies.comments'],
      dtype='object')

In [10]:
df.to_csv(f'comments-{VIDEO_ID}.csv',index=False)

In [11]:
data=pd.read_csv(f'comments-{VIDEO_ID}.csv')

In [12]:
data.head()

,kind,etag,id,channelId,videoId,kind.1,etag.1,id.1,channelId.1,videoId.1,...,authorChannelId,canRate,viewerRating,likeCount,publishedAt,updatedAt,canReply,totalReplyCount,isPublic,replies.comments
0,youtube#commentThread,3mBUGIip4cJzBs1R2FJcoAoD_-A,UgyiVIVM7XBnRFfhvr54AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,youtube#comment,vslFAPPvV2pErpmlic8EO_MInuQ,UgyiVIVM7XBnRFfhvr54AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,...,UCUsN5ZwHx2kILm84-jPDeXw,True,none,10393,2019-09-03T18:43:15Z,2019-09-03T18:43:15Z,True,92,True,"[{'kind': 'youtube#comment', 'etag': '4J3UO6Ud..."
1,youtube#commentThread,UDsqMzCGORU_qhUh-xVRrI773TU,UgzOzDX1jT_2jCzSkVp4AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,youtube#comment,ItUdfEZO9OQVD37DJBPSoh1WOQ4,UgzOzDX1jT_2jCzSkVp4AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,...,UC61bOqWqm7n713umLk8_6fQ,True,none,0,2023-11-26T14:27:20Z,2023-11-26T14:27:53Z,True,0,True,NaN
2,youtube#commentThread,KA3N0MGLE3SD711iH19SPWqQmI4,Ugxkt5ag1bZQT4H1Lkd4AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,youtube#comment,dMcc5qXlNuRfWO3kvqhMGZoaJHU,Ugxkt5ag1bZQT4H1Lkd4AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,...,UC9fJlw2kDrpO0K-tnTr96YQ,True,none,0,2023-11-24T02:04:02Z,2023-11-24T02:04:02Z,True,0,True,NaN
3,youtube#commentThread,hWbSgNujOAquGc2wt685Xv7Ueho,UgxS9faNwb01M7cLvGl4AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,youtube#comment,ZMBw3ZeRC9uAbEGOqBleF1UlVcs,UgxS9faNwb01M7cLvGl4AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,...,UCStbrsZziGZXtvyxvQzMjfQ,True,none,0,2023-11-22T08:43:29Z,2023-11-22T08:43:29Z,True,0,True,NaN
4,youtube#commentThread,ilfrCMU3RUUDcY9nlvJ63Y-JqmY,UgwWJsvEnp0HNoqezCV4AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,youtube#comment,FuEi4FmXmvory_qNhHIfelcUU7E,UgwWJsvEnp0HNoqezCV4AaABAg,UCUsN5ZwHx2kILm84-jPDeXw,LAOoF2gyQaA,...,UCkBAUYfxGnockoTDMC3l46g,True,none,0,2023-11-20T10:15:29Z,2023-11-20T10:15:29Z,True,0,True,NaN


In [13]:
data['textOriginal']

0     The Roast of Alec Baldwin premieres September ...
1     7:00 shaq and kevin laughing at snoop was the ...
2     Bieber took it like a champ. They were brutal ...
3                                 That Hannibal Guy 😭😭😭
4                                     Pete's last line😂
                            ...                        
95             Lol they need Eminem in this roast thing
96    What's happening to Americans I mean 9/11 is a...
97    Always back here and enjoy it like the first t...
98                                                    😊
99                                 king joeffrey of pop
Name: textOriginal, Length: 100, dtype: object

In [14]:
text_df=data.drop(['kind.1', 'etag.1', 'id.1', 'channelId.1', 'videoId.1', 'kind', 'etag', 'id',
       'channelId', 'videoId', 'textDisplay','authorDisplayName', 'authorProfileImageUrl', 'authorChannelUrl',
       'authorChannelId', 'canRate', 'viewerRating', 'likeCount',
       'publishedAt', 'updatedAt', 'canReply', 'totalReplyCount', 'isPublic','replies.comments'],axis=1)
text_df.head()

,textOriginal
0,The Roast of Alec Baldwin premieres September ...
1,7:00 shaq and kevin laughing at snoop was the ...
2,Bieber took it like a champ. They were brutal ...
3,That Hannibal Guy 😭😭😭
4,Pete's last line😂


In [15]:
print(text_df['textOriginal'].iloc[0],"\n")
print(text_df['textOriginal'].iloc[1],"\n")
print(text_df['textOriginal'].iloc[2],"\n")
print(text_df['textOriginal'].iloc[3],"\n")
print(text_df['textOriginal'].iloc[4],"\n")

The Roast of Alec Baldwin premieres September 15 on Comedy Central. See who will be roasting Alec here: 
https://www.youtube.com/watch?v=ePp-8vOmmdk 

7:00 shaq and kevin laughing at snoop was the best😂😂😂 

Bieber took it like a champ. They were brutal to him 😅😂 

That Hannibal Guy 😭😭😭 

Pete's last line😂 



**Data labelling**

In [16]:
nltk.download('vader_lexicon')
sentiments = SentimentIntensityAnalyzer()
text_df["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in text_df["textOriginal"]]
text_df["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in text_df["textOriginal"]]
text_df["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in text_df["textOriginal"]]
text_df['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in text_df["textOriginal"]]
score = text_df["Compound"].values
sentiment = []
for i in score:
    if i >= 0.05 :
        sentiment.append('Positive')
    elif i <= -0.05 :
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')
text_df["Sentiment"] = sentiment
text_df.head()

[nltk_data] Downloading package vader_lexicon to C:\Users\Mayuri
[nltk_data]     chauhan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,textOriginal,Positive,Negative,Neutral,Compound,Sentiment
0,The Roast of Alec Baldwin premieres September ...,0.122,0.000,0.878,0.3612,Positive
1,7:00 shaq and kevin laughing at snoop was the ...,0.262,0.000,0.738,0.4939,Positive
2,Bieber took it like a champ. They were brutal ...,0.316,0.232,0.452,0.1280,Positive
3,That Hannibal Guy 😭😭😭,0.000,0.000,1.000,0.0000,Neutral
4,Pete's last line😂,0.000,0.000,1.000,0.0000,Neutral


**Finaldata**

In [17]:
data2=text_df.drop(['Positive','Negative','Neutral','Compound'],axis=1)
data2.head()

,textOriginal,Sentiment
0,The Roast of Alec Baldwin premieres September ...,Positive
1,7:00 shaq and kevin laughing at snoop was the ...,Positive
2,Bieber took it like a champ. They were brutal ...,Positive
3,That Hannibal Guy 😭😭😭,Neutral
4,Pete's last line😂,Neutral


In [18]:
stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer() 
snowball_stemer = SnowballStemmer(language="english")
lzr = WordNetLemmatizer()

In [19]:
def text_processing(text):   
    # convert text into lowercase
    text = text.lower()

    # remove new line characters in text
    text = re.sub(r'\n',' ', text)
    
    # remove punctuations from text
    text = re.sub('[%s]' % re.escape(punctuation), "", text)
    
    # remove references and hashtags from text
    text = re.sub("^a-zA-Z0-9$,.", "", text)
    
    # remove multiple spaces from text
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # remove special characters from text
    text = re.sub(r'\W', ' ', text)

    text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])
    
    # stemming using porter stemmer from nltk package - msh a7sn 7aga - momken: lancaster, snowball
    # text=' '.join([porter_stemmer.stem(word) for word in word_tokenize(text)])
    # text=' '.join([lancaster_stemmer.stem(word) for word in word_tokenize(text)])
    # text=' '.join([snowball_stemer.stem(word) for word in word_tokenize(text)])
    
    # lemmatizer using WordNetLemmatizer from nltk package
    text=' '.join([lzr.lemmatize(word) for word in word_tokenize(text)])

    return text

In [20]:
data_copy = data2.copy()
data_copy.textOriginal = data_copy.textOriginal.apply(lambda text: text_processing(text))

In [21]:
le = LabelEncoder()
data_copy['Sentiment'] = le.fit_transform(data_copy['Sentiment'])

In [22]:

processed_data = {
    'Sentence':data_copy.textOriginal,
    'Sentiment':data_copy['Sentiment']
}

processed_data = pd.DataFrame(processed_data)
processed_data.head()

,Sentence,Sentiment
0,roast alec baldwin premiere september 15 comed...,2
1,700 shaq kevin laughing snoop best,2
2,bieber took like champ brutal,2
3,hannibal guy,1
4,petes last line,1


In [23]:
df_neutral = processed_data[(processed_data['Sentiment']==1)] 
df_negative = processed_data[(processed_data['Sentiment']==0)]
df_positive = processed_data[(processed_data['Sentiment']==2)]

# upsample minority classes
df_negative_upsampled = resample(df_negative, 
                                 replace=True,    
                                 n_samples= 205, 
                                 random_state=42)  

df_neutral_upsampled = resample(df_neutral, 
                                 replace=True,    
                                 n_samples= 205, 
                                 random_state=42)  


# Concatenate the upsampled dataframes with the neutral dataframe
final_data = pd.concat([df_negative_upsampled,df_neutral_upsampled,df_positive])

In [24]:
final_data['Sentiment'].value_counts()

Sentiment
0    205
1    205
2     39
Name: count, dtype: int64

In [25]:
corpus = []
for sentence in final_data['Sentence']:
    corpus.append(sentence)
corpus[0:5]

['shaquille fact word handsome arabic thats b',
 'pete davidson dad joke killed',
 'cut hannibal buress part roast youre trash cc',
 'prison easier making blueberry scone',
 'shaquille fact word handsome arabic thats b']

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()
y = final_data.iloc[:, -1].values

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [28]:
from sklearn.naive_bayes import GaussianNB

In [29]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [30]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)

In [31]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[63,  0,  0],
       [ 8, 50,  1],
       [ 5,  2,  6]], dtype=int64)

In [32]:
nb_score = accuracy_score(y_test, y_pred)
nb_score

0.8814814814814815